In [1]:
import selenium
from selenium import webdriver
from selenium.webdriver.edge.service import Service as EdgeService
from webdriver_manager.microsoft import EdgeChromiumDriverManager
from selenium.webdriver.common.by import By  
import pandas as pd
from datetime import datetime
import xlsxwriter
import time
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

import os
import sys
import requests

In [4]:
def extract_image_links(url):
    
    parent = driver.find_element(By.CLASS_NAME, "flex.sm\\:block.gap-4")
    child_element = parent.find_element(By.TAG_NAME, "a")
    try:
        link = child_element.get_attribute("href")
    except:
        link = "a/A/A/A/NONE/NONE/NONE/NONE"
    brand, model = link.split('/')[5:7]
    a = brand
    b = model
    print(a, b)
    
    truc = []
    c = 0
    while c<5:
        photos = driver.find_elements(By.CLASS_NAME, "img.basis-80.w-80.sm\\:w-full.h-50.text-center.relative.loading")
        print(len(photos))

        for photo in photos:
            phot = photo.find_element(By.CLASS_NAME, "w-full.object-cover.h-50")
            phot = phot.get_attribute("src")
            truc.append(phot)
        print("got links")
        pgsv = driver.find_element(By.CLASS_NAME, "pgsuiv")
        pgsv = pgsv.find_element(By.TAG_NAME, "a")
        print("got next page")
        try:
            element = driver.find_element(By.CSS_SELECTOR, "a.relative.inline-flex.items-center.border.border-gray-300.bg-white.px-3.5.py-3.5.text-sm.font-medium.hover:bg-gray-50.focus:z-20.cursor-pointer")
            driver.execute_script("arguments[0].click();", element)
        except:
            nv_url = url + "?p=" + str(c+1)
            driver.get(nv_url)
        print("next page")
        c+=1
            
    i=0
    for link in truc:
        # Téléchargez et enregistrez chaque image dans le dossier "car_images"
        file_name = f"{a}-{b}-{i+1}.jpg"
        save_path = os.path.join("cars_project", file_name)
        download_image(link, save_path)
        sys.stdout.write(f"\rDownloaded {i+1}/{len(truc)} images")
        i+=1
    print("\nDone!")

def download_image(url, save_path):
    response = requests.get(url, stream=True)
    if response.status_code == 200:
        with open(save_path, 'wb') as file:
            for chunk in response.iter_content(1024):
                file.write(chunk)
    else:
        print(f"Error {response.status_code}: unable to download image from {url}")

In [6]:
driver = webdriver.Edge()

liste_url = [
    "https://www.paruvendu.fr/a/voiture-occasion/peugeot/208/",
    "https://www.paruvendu.fr/a/voiture-occasion/renault/clio/",
    "https://www.paruvendu.fr/a/voiture-occasion/dacia/sandero/",
    "https://www.paruvendu.fr/a/voiture-occasion/citroen/c3/",
    "https://www.paruvendu.fr/a/voiture-occasion/peugeot/2008/",
    "https://www.paruvendu.fr/a/voiture-occasion/renault/captur/",
    "https://www.paruvendu.fr/a/voiture-occasion/peugeot/3008/",
    "https://www.paruvendu.fr/a/voiture-occasion/toyota/yaris/",
    "https://www.paruvendu.fr/a/voiture-occasion/citroen/c4/",
    "https://www.paruvendu.fr/a/voiture-occasion/volkswagen/golf/",
    "https://www.paruvendu.fr/a/voiture-occasion/renault/megane/",
    "https://www.paruvendu.fr/a/voiture-occasion/ford/fiesta/",
    "https://www.paruvendu.fr/a/voiture-occasion/peugeot/308/",
    "https://www.paruvendu.fr/a/voiture-occasion/renault/twingo/",
    "https://www.paruvendu.fr/a/voiture-occasion/volkswagen/polo/"
]

nb_model = 1
for url in liste_url:
    driver.get(url)
    liste = pd.DataFrame()
    print("Driver créé")
    
    if nb_model == 1:
        cookie = driver.find_element(By.XPATH, "//button[@onclick='cmp_pv.cookie.saveConsent(true);']")
        cookie.click()

    os.makedirs("cars_project", exist_ok=True)

    extract_image_links(url)
    nb_model+=1

driver.quit()

Driver créé
dacia sandero
25
got links
got next page
next page
25
got links
got next page
next page
25
got links
got next page
next page
25
got links
got next page
next page
25
got links
got next page
next page
Downloaded 125/125 images
Done!
Driver créé
citroen c3
25
got links
got next page
next page
25
got links
got next page
next page
25
got links
got next page
next page
25
got links
got next page
next page
25
got links
got next page
next page
Downloaded 125/125 images
Done!
Driver créé
peugeot 2008
25
got links
got next page
next page
25
got links
got next page
next page
25
got links
got next page
next page
25
got links
got next page
next page
25
got links
got next page
next page
Downloaded 125/125 images
Done!
Driver créé
renault captur
25
got links
got next page
next page
25
got links
got next page
next page
25
got links
got next page
next page
25
got links
got next page
next page
25
got links
got next page
next page
Downloaded 125/125 images
Done!
Driver créé
peugeot 3008
25
got

## Preprocessing et modèles

In [45]:
import os
import pickle
import pandas as pd
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.preprocessing.image import ImageDataGenerator, load_img, img_to_array
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelBinarizer
from sklearn.utils import class_weight
from tensorflow.keras.utils import to_categorical
import numpy as np
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt

# Set the directory and parameters
data_dir = r'C:\Users\user\Documents\cars_project'
img_width, img_height = 480, 360
batch_size = 32
epochs = 35

# Load filenames and extract labels
filenames = os.listdir(data_dir)
labels = [filename.split('-')[:2] for filename in filenames]
labels = pd.DataFrame(labels, columns=['brand', 'model'])
labels['filename'] = filenames


# Kmeans 

k = 15
kmeans = KMeans(n_clusters = k, random_state=42)
kmeans.fit(filenames)

labels = kmeans.labels_
centers = kmeans.cluster_centers_

#End of Kmeans


# Split data into training and validation sets
train_filenames, val_filenames, train_labels, val_labels = train_test_split(
    labels['filename'], labels['model'], test_size=0.2, random_state=42
)

# Create data generators for training and validation sets
train_datagen = ImageDataGenerator(
    rescale=1. / 255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True
)

validation_datagen = ImageDataGenerator(rescale=1. / 255)

train_generator = train_datagen.flow_from_dataframe(
    dataframe=labels.loc[labels['filename'].isin(train_filenames)],
    directory=data_dir,
    x_col='filename',
    y_col='model',
    color_mode = 'grayscale',
    target_size=(img_width, img_height),
    class_mode='categorical',
    batch_size=batch_size,
    shuffle=True,
    seed=42
)

validation_generator = validation_datagen.flow_from_dataframe(
    dataframe=labels.loc[labels['filename'].isin(val_filenames)],
    directory=data_dir,
    x_col='filename',
    y_col='model',
    color_mode = 'grayscale',
    target_size=(img_width, img_height),
    class_mode='categorical',
    batch_size=batch_size,
    shuffle=False,
    seed=42
)

# Get the number of unique car models (classes)
num_classes = len(labels['model'].unique())

# Define the CNN model
model = Sequential()
model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(img_width, img_height, 1)))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes, activation='softmax'))

# Compile the model
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

# Create a label binarizer for one-hot encoding
label_binarizer = LabelBinarizer()
train_labels_one_hot = label_binarizer.fit_transform(train_labels)
val_labels_one_hot = label_binarizer.transform(val_labels)

# Train the model using fit with custom data generators
train_generator.reset()
validation_generator.reset()

model.fit(
    train_generator,
    epochs=epochs,
    validation_data=validation_generator,
    callbacks=[tf.keras.callbacks.EarlyStopping(patience=10, restore_best_weights=True)]
)


# Save the model (optional)
model.save('car_model_classifier.h5')

# Save the label binarizer (to be used during prediction)
with open('label_binarizer.pkl', 'wb') as f:
    pickle.dump(label_binarizer, f)


Found 1456 validated image filenames belonging to 15 classes.
Found 365 validated image filenames belonging to 15 classes.
Epoch 1/35
46/46 [==============================] - 195s 4s/step - loss: 4.6944 - accuracy: 0.0742 - val_loss: 2.6572 - val_accuracy: 0.0959
Epoch 2/35
46/46 [==============================] - 196s 4s/step - loss: 2.6698 - accuracy: 0.0790 - val_loss: 2.6202 - val_accuracy: 0.0904
Epoch 3/35
46/46 [==============================] - 222s 5s/step - loss: 2.6404 - accuracy: 0.1058 - val_loss: 2.6031 - val_accuracy: 0.1014
Epoch 4/35
46/46 [==============================] - 293s 6s/step - loss: 2.6275 - accuracy: 0.1010 - val_loss: 2.5832 - val_accuracy: 0.1123
Epoch 5/35
46/46 [==============================] - 324s 7s/step - loss: 2.5998 - accuracy: 0.1174 - val_loss: 2.5645 - val_accuracy: 0.1288
Epoch 6/35
46/46 [==============================] - 357s 8s/step - loss: 2.5657 - accuracy: 0.1291 - val_loss: 2.5508 - val_accuracy: 0.1342
Epoch 7/35
46/46 [=============

KeyboardInterrupt: 

In [38]:
import tensorflow as tf
import pickle
import numpy as np
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.resnet50 import preprocess_input


with open('label_binarizer.pkl', 'rb') as f:
    label_binarizer = pickle.load(f)

# Replace 'path_to_your_image.jpg' with the actual path to the image you want to predict
img_path = r"C:\Users\user\Documents\cars_project\peugeot-2008-85.jpg"

# Load and preprocess the image
img = image.load_img(img_path, target_size=(250, 190), color_mode = 'grayscale')
x = image.img_to_array(img)
x = np.expand_dims(x, axis=0)

# Make a prediction
predictions = model.predict(x)

# Get the index of the predicted class
predicted_class_idx = np.argmax(predictions[0])

# Get the predicted car model
predicted_model = label_binarizer.classes_[predicted_class_idx]

print(f"Predicted car model: {predicted_model}")

1/1 [==============================] - 0s 42ms/step
Predicted car model: 2008


# Object detection

In [51]:
import cv2

thres = 0.50  # Threshold to detect objects

# Load class names from coco.names file
classNames = []
classFile = 'coco.names'
with open(classFile, 'rt') as f:
    classNames = f.read().rstrip('\n').split('\n')

# Load the pre-trained model
configPath = 'ssd_mobilenet_v3_large_coco_2020_01_14.pbtxt'
weightsPath = 'frozen_inference_graph.pb'

net = cv2.dnn_DetectionModel(weightsPath, configPath)
net.setInputSize(320, 320)
net.setInputScale(1.0 / 127.5)
net.setInputMean((127.5, 127.5, 127.5))
net.setInputSwapRB(True)

def detect_cars(image_path):
    img = cv2.imread(image_path)

    # Check if the image is loaded successfully
    if img is None:
        print("Error: Image not found or invalid image path.")
        return [], None

    img = cv2.resize(img, (700, 500))

    classIds, confs, bbox = net.detect(img, confThreshold=thres)

    car_rectangles = []

    if len(classIds) != 0:
        for classId, confidence, box in zip(classIds.flatten(), confs.flatten(), bbox):
            # Ensure classId is within the valid range and corresponds to a car
            if 0 <= classId < len(classNames) and classNames[classId - 1].lower() == 'car':
                car_rectangles.append(box)
                cv2.rectangle(img, box, color=(0, 255, 0), thickness=2)
                cv2.putText(img, classNames[classId - 1].upper(), (box[0] + 10, box[1] + 30),
                            cv2.FONT_HERSHEY_COMPLEX, 1, (0, 255, 0), 2)
                cv2.putText(img, str(round(confidence * 100, 2)), (box[0] + 200, box[1] + 30),
                            cv2.FONT_HERSHEY_COMPLEX, 1, (0, 255, 0), 2)
            else:
                print(f"Invalid classId: {classId}")

    return car_rectangles, img

# Replace 'path_to_your_image.jpg' with the actual path to the image you want to detect cars in
image_path = r"C:\Users\user\Documents\cars_project\peugeot-2008-8.jpg"
car_rectangles, img = detect_cars(image_path)
print("Car rectangles:", car_rectangles)

# Display the image with detected objects
cv2.imshow('Image with Detected Cars', img)
cv2.waitKey(0)
cv2.destroyAllWindows()


Car rectangles: [array([  0,  89, 662, 297])]
